In [50]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('data_training_roberta.csv')
df.head()

,index,roberta_neg_post_body,roberta_neu_post_body,roberta_pos_post_body,roberta_compound_post_body,roberta_neg_post_title,roberta_neu_post_title,roberta_pos_post_title,roberta_compound_post_title,subreddit,post_id,post_title,post_score,post_url,post_comms_num,post_body,post_timestamp
0,1,0.171448,0.768685,0.059867,-0.111581,0.013251,0.928738,0.058012,0.044761,television,1gmlvja,watch recommend week november 08 2024,51,https://www.reddit.com/r/television/comments/1...,406,comment sort new default feel free describe sh...,2024-11-08 16:00:12
1,2,0.101025,0.739353,0.159622,0.058597,0.268124,0.694113,0.037763,-0.230361,television,1gpuzer,kiernan shipka say know go mad man despite eff...,1411,https://people.com/kiernan-shipka-says-she-kne...,74,""" I know brain body decide remember like audit...",2024-11-12 20:45:23
2,3,0.031195,0.229534,0.739270,0.708075,0.008140,0.296943,0.694917,0.686777,television,1gpkns9,"jim gaffigan end time tim walz ' snl ' "" I fee...",943,https://www.hollywoodreporter.com/tv/tv-news/j...,97,""" it expectation maya going continue entire ti...",2024-11-12 13:25:57
3,4,0.122717,0.428424,0.448859,0.326142,0.013590,0.429056,0.557354,0.543763,television,1gq3tw7,hbo series oz unforgettable character tv,62,https://www.reddit.com/r/television/comments/1...,36,ryan oreilly vern schillinger simon adebisi to...,2024-11-13 03:35:16
4,5,0.065585,0.878166,0.056249,-0.009336,0.073045,0.879932,0.047023,-0.026022,television,1gq3oj3,st denis medical series premiere discussion,21,https://www.reddit.com/r/television/comments/1...,6,st denis medical premise staff oregon understa...,2024-11-13 03:27:24


In [51]:
import torch
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split

In [ ]:

df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

tokenized_train = tokenizer(df_train["post_body"].values.tolist(), padding=True, truncation=True, return_tensors="pt")
tokenized_val = tokenizer(df_val["post_body"].values.tolist(), padding=True, truncation=True, return_tensors="pt")

tokenized_train = {k: v.to(device) for k, v in tokenized_train.items()}
tokenized_val = {k: v.to(device) for k, v in tokenized_val.items()}

In [ ]:
with torch.no_grad():
    hidden_train = model(**tokenized_train)
    hidden_val = model(**tokenized_val)

# Get only the [CLS] hidden states (first token)
cls_train = hidden_train.last_hidden_state[:, 0, :]
cls_val = hidden_val.last_hidden_state[:, 0, :]

x_train = cls_train.cpu().numpy()
x_val = cls_val.cpu().numpy()

y_train = df_train["roberta_compound_post_body"].values
y_val = df_val["roberta_compound_post_body"].values

RuntimeError: Invalid buffer size: 16.85 GB

In [ ]:
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")

x_train shape: (8, 768), y_train shape: (11504,)
x_val shape: (8, 768), y_val shape: (2877,)


In [ ]:
reg_model = LinearRegression()
reg_model.fit(x_train, y_train)

y_pred = reg_model.predict(x_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = reg_model.score(x_val, y_val)
print(f"R^2 score: {r2}")

ValueError: Found input variables with inconsistent numbers of samples: [8, 11504]